In [13]:
# default values
role_param = ""
output_bucket_path = ""

In [44]:
%%time
import boto3
from time import gmtime, strftime
from sagemaker import image_uris
processing_job_name = f"processing-job-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
region = "us-east-1"
role = role_param
client = boto3.client("sagemaker", region_name=region)
deploy_amt_model = True
output_prefix = "local-pipeline"


CPU times: user 5.88 ms, sys: 1.38 ms, total: 7.26 ms
Wall time: 6.5 ms


In [45]:
image_uri = image_uris.retrieve(
        framework="xgboost",
        region="us-east-1",
        version="1.0-1",
        py_version="py3",
        instance_type="ml.m5.large",
)

In [46]:
create_processing_params = {
   "AppSpecification": {
        'ImageUri': image_uri
   },
   "ProcessingInputs": [
      {
         "InputName": "abalone.train",
         "S3Input": {
            "LocalPath": "/opt/ml/processing/input",
            "S3DataDistributionType": "FullyReplicated",
            "S3InputMode": "File",
            'S3DataType': "S3Prefix",
            "S3Uri": "s3://kris-mlops-utilities-test/abalone_data/train"
         }
      }
   ],
   "StoppingCondition": {
        'MaxRuntimeInSeconds': 123
   },
   "ProcessingJobName": processing_job_name,
   "ProcessingOutputConfig": {
      "Outputs": [
         {
            "OutputName": "preprocessing_output",
            "S3Output": {
               "LocalPath": "/opt/ml/processing/output",
               "S3Uri": f"s3://{output_bucket_path}/preprocessing",
               "S3UploadMode": "EndOfJob"
            }
         }
      ]
   },
   "ProcessingResources": {
      "ClusterConfig": {
         "InstanceCount": 1,
         "InstanceType": "ml.m5.large",
         "VolumeSizeInGB": 5
      }
   },
   "RoleArn": role,
}

In [48]:
processing_job_name

'processing-job-2023-02-07-14-07-04'

In [49]:
print(f"Creating a training job with name: {processing_job_name}. It will take between 5 and 6 minutes to complete.")
client.create_processing_job(**create_processing_params)